In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Reading the data
df = pd.read_excel("../insights/bird_speed_analysis.xlsx")

# Create a scatter mapbox figure
fig = px.scatter_mapbox(
    df,
    lat="start_lat",
    lon="start_long",
    color="speed_kmph",
    size="speed_kmph",
    hover_name="individual-local-identifier",
    hover_data=["distance_km", "speed_kmph"],
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=15,
    zoom=10,
    mapbox_style="open-street-map"
)

# Add lines for the path
fig.add_trace(go.Scattermapbox(
    mode="lines",
    lon=df["start_long"],
    lat=df["start_lat"],
    line=dict(width=2, color='blue'),
    name="Path"
))

# Calculate midpoints and angles
midpoints = pd.DataFrame({
    "lon": (df["start_long"] + df["start_long"].shift(-1)) / 2,
    "lat": (df["start_lat"] + df["start_lat"].shift(-1)) / 2,
    "angle": np.degrees(np.arctan2(df["start_lat"].shift(-1) - df["start_lat"], df["start_long"].shift(-1) - df["start_long"]))
})

# Remove the last row (since it will have NaN values from the shift operation)
midpoints = midpoints[:-1]

# Add arrows to indicate direction
fig.add_trace(go.Scattermapbox(
    mode="markers",
    lon=midpoints["lon"],
    lat=midpoints["lat"],
    marker=go.scattermapbox.Marker(
        size=40,
        symbol="triangle-up",
        angle=midpoints["angle"],
        color="red"
    ),
    showlegend=False
))

# Update layout
fig.update_layout(
    title="Bird Movement with Direction and Speed Indication",
    margin={"r":0,"t":0,"l":0,"b":0},
    showlegend=True
)

fig.show()


ValueError: Value of 'hover_name' is not the name of a column in 'data_frame'. Expected one of ['individual-local-identifier', 'start_time', 'end_time', 'start_lat', 'start_long', 'end_lat', 'end_long', 'distance_km', 'time_diff_hours', 'speed_kmph'] but received: bird_id

In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import contextily as ctx

# Load data into a DataFrame
df = pd.read_excel("../insights/bird_speed_analysis.xlsx")

# Create GeoDataFrame with LineString for each journey
gdf = gpd.GeoDataFrame(
    df,
    geometry=df.apply(lambda row: LineString([(row['start_long'], row['start_lat']), 
                                              (row['end_long'], row['end_lat'])]), axis=1),
    crs="EPSG:4326"
)

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, column='speed_kmph', legend=True, cmap='coolwarm', linewidth=3)

# Add basemap with OpenStreetMap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Annotate the direction with arrows
for idx, row in gdf.iterrows():
    ax.annotate('', xy=(row['end_long'], row['end_lat']), 
                xytext=(row['start_long'], row['start_lat']),
                arrowprops=dict(facecolor='black', shrink=0.05))

# Set labels
ax.set_title('Bird Movement with Speed and Direction')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

c:\Users\visha\AppData\Local\Programs\Python\Python312\Lib\site-packages\contextily\tile.py:645: UserWarning: The inferred zoom level of 29 is not valid for the current tile provider (valid zooms: 0 - 19).
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import plotly.graph_objects as go


# Load data into a DataFrame
df = pd.read_excel("./insights/bird_speed_analysis.xlsx")

df = pd.DataFrame(df)

# Create Plotly scatter plot with arrows for direction
fig = go.Figure()

# Scatter points for the start locations
fig.add_trace(go.Scattergeo(
    lat=df['start_lat'],
    lon=df['start_long'],
    mode='markers',
    marker=dict(
        size=8,
        color=df['speed_kmph'],
        colorscale='Viridis',
        colorbar=dict(title='Speed (km/h)')
    ),
    hoverinfo='text',
    hovertext=df.apply(lambda row: (
        f"Timestamp: {row['timestamp']}<br>"
        f"Speed: {row['speed_kmph']} km/h<br>"
        f"Start: ({row['start_lat']}, {row['start_long']})<br>"
        f"End: ({row['end_lat']}, {row['end_long']})"
    ), axis=1)
))

# Add lines with arrows to show the direction
for _, row in df.iterrows():
    fig.add_trace(go.Scattergeo(
        lat=[row['start_lat'], row['end_lat']],
        lon=[row['start_long'], row['end_long']],
        mode='lines',
        line=dict(width=2, color='blue'),
        showlegend=False
    ))

# Update layout to enable zoom and improve appearance
fig.update_layout(
    geo=dict(
        scope='world',
        projection=dict(type='equirectangular'),
        showland=True,
        landcolor="rgb(243, 243, 243)",
        subunitcolor="rgb(217, 217, 217)",
        showcountries=True,
        showocean=True,
        oceancolor="rgb(204, 255, 255)",
        showlakes=True,
        lakecolor="rgb(204, 255, 255)",
        showrivers=True,
        rivercolor="rgb(204, 255, 255)",
        resolution=50,
        lataxis=dict(range=[min(df['start_lat']) - 5, max(df['end_lat']) + 5]),
        lonaxis=dict(range=[min(df['start_long']) - 5, max(df['end_long']) + 5])
    ),
    title='Bird Speed Analysis with Direction and Hover Information',
    showlegend=False,
    width=1200,
    height=800   
)


fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load data into a DataFrame
df = pd.read_excel("./insights/bird_speed_analysis.xlsx")

# Create Plotly scatter plot with arrows for direction
fig = go.Figure()

# Scatter points for the start locations
fig.add_trace(go.Scattergeo(
    lat=df['start_lat'],
    lon=df['start_long'],
    mode='markers',
    marker=dict(
        size=8,
        color=df['speed_kmph'],
        colorscale='Viridis',
        colorbar=dict(title='Speed (km/h)')
    ),
    hoverinfo='text',
    hovertext=df.apply(lambda row: (
        f"Timestamp: {row['timestamp']}<br>"
        f"Speed: {row['speed_kmph']} km/h<br>"
        f"Start: ({row['start_lat']}, {row['start_long']})<br>"
        f"End: ({row['end_lat']}, {row['end_long']})"
    ), axis=1)
))

# Add lines with arrows to show the direction and color by speed
for _, row in df.iterrows():
    speed = row['speed_kmph']
    color = f"rgb({255 * (1 - speed / 60):.0f}, {255 * (speed / 60):.0f}, 150)"  # Adjust the divisor for scaling speed
    
    fig.add_trace(go.Scattergeo(
        lat=[row['start_lat'], row['end_lat']],
        lon=[row['start_long'], row['end_long']],
        mode='lines+markers',
        line=dict(width=2, color=color, dash='dot'),
        marker=dict(size=10, symbol='arrow-up'),  # Changed symbol to 'arrow-right'
        showlegend=False
    ))

# Update layout to enable zoom and improve appearance
fig.update_layout(
    geo=dict(
        scope='world',
        projection=dict(type='equirectangular'),
        showland=True,
        landcolor="rgb(243, 243, 243)",
        subunitcolor="rgb(217, 217, 217)",
        showcountries=True,
        showocean=True,
        oceancolor="rgb(204, 255, 255)",
        showlakes=True,
        lakecolor="rgb(204, 255, 255)",
        showrivers=True,
        rivercolor="rgb(204, 255, 255)",
        resolution=50,
        lataxis=dict(range=[min(df['start_lat']) - 5, max(df['end_lat']) + 5]),
        lonaxis=dict(range=[min(df['start_long']) - 5, max(df['end_long']) + 5])
    ),
    title='Bird Speed Analysis with Direction and Hover Information',
    showlegend=False,
    width=1200,
    height=800   
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Load data into a DataFrame
df = pd.read_excel("./insights/bird_speed_analysis.xlsx")

# Create Plotly scatter plot
fig = go.Figure()

# Scatter points for the start locations
fig.add_trace(go.Scattergeo(
    lat=df['start_lat'],
    lon=df['start_long'],
    mode='markers',
    marker=dict(
        size=8,
        color=df['speed_kmph'],
        colorscale='Viridis',
        colorbar=dict(title='Speed (km/h)')
    ),
    hoverinfo='text',
    hovertext=df.apply(lambda row: (
        f"Timestamp: {row['timestamp']}<br>"
        f"Speed: {row['speed_kmph']} km/h<br>"
        f"Start: ({row['start_lat']}, {row['start_long']})<br>"
        f"End: ({row['end_lat']}, {row['end_long']})"
    ), axis=1)
))

# Add arrows to show direction using SVG path
shapes = []
for _, row in df.iterrows():
    # Calculate direction vector
    dx = row['end_long'] - row['start_long']
    dy = row['end_lat'] - row['start_lat']
    angle = np.arctan2(dy, dx) * 180 / np.pi
    
    # Define SVG path for arrow
    arrow_path = (
        f"M {row['start_long']} {row['start_lat']} "
        f"L {row['end_long']} {row['end_lat']} "
        f"L {row['end_long'] - 0.5 * dx - 0.3 * dy} {row['end_lat'] - 0.5 * dy + 0.3 * dx} "
        f"M {row['end_long']} {row['end_lat']} "
        f"L {row['end_long'] - 0.5 * dx + 0.3 * dy} {row['end_lat'] - 0.5 * dy - 0.3 * dx}"
    )
    
    # Add shape for arrow
    shapes.append(dict(
        type='path',
        path=arrow_path,
        line=dict(
            color='rgba(255, 0, 0, 0.6)',  # Adjust color if needed
            width=2
        ),
        opacity=0.6
    ))

# Update layout
fig.update_layout(
    geo=dict(
        scope='world',
        projection=dict(type='equirectangular'),
        showland=True,
        landcolor="rgb(243, 243, 243)",
        subunitcolor="rgb(217, 217, 217)",
        showcountries=True,
        showocean=True,
        oceancolor="rgb(204, 255, 255)",
        showlakes=True,
        lakecolor="rgb(204, 255, 255)",
        showrivers=True,
        rivercolor="rgb(204, 255, 255)",
        resolution=50,
        lataxis=dict(range=[min(df['start_lat']) - 5, max(df['end_lat']) + 5]),
        lonaxis=dict(range=[min(df['start_long']) - 5, max(df['end_long']) + 5])
    ),
    title='Bird Speed Analysis with Direction and Hover Information',
    showlegend=False,
    width=1200,
    height=800,
    shapes=shapes  # Add shapes to the layout
)

fig.show()
